Setup

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Data

In [0]:
# To load in our data

#data = pd.read_csv('drive/My Drive/CS 598PS Final Project/data/nfl_row_processed_2_norm.csv')
#target = pd.read_csv('drive/My Drive/CS 598PS Final Project/nfl_bigbowl_data_mod_target.csv', header=None)

In [0]:
# To load in the data from Kaggle-NB

# data = pd.read_csv('drive/My Drive/CS 598PS Final Project/data/nfl_enhanced_db.csv', header=0, index_col=0)
# target = data['Yards']
# data.drop(['GameId','PlayId','Yards'], axis=1, inplace=True)

In [0]:
data = pd.read_csv('drive/My Drive/CS 598PS Final Project/data/combined_data_smarter_2.csv', header=0, index_col=0)
target = data['Yards']
data.drop(['Yards'], axis=1, inplace=True)

In [6]:
data

,GameClock_in_seconds,in_own_half,home_team_has_poss,Formation_ACE,Formation_EMPTY,Formation_I_FORM,Formation_JUMBO,Formation_PISTOL,Formation_SHOTGUN,Formation_SINGLEBACK,Formation_WILDCAT,play_dir_is_left,TimeDelta,Quarter_1,Quarter_2,Quarter_3,Quarter_4,Quarter_5,Down_1,Down_2,Down_3,Down_4,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,RusherDir,RusherS,RusherA,RusherX,RusherY,v_horizontal,v_vertical,PlayerHeight_dense,PlayerAge,WindSpeed_dense,Dir_sin,Dir_cos,diffScoreBeforePlay,X,Y,S,A,Dis,Orientation,Dir,YardLine,Quarter,Down,Distance,DefendersInTheBox
0,854,True,True,0,0,0,0,0,1,0,0,True,1.0,1,0,0,0,0,0,0,1,0,-33.75,0,1,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,245.74,3.63,3.35,78.75,30.53,3.309436,1.491487,70,25.596251,8.0,-0.911690,-0.410878,0,78.75,30.53,3.63,3.35,0.38,161.98,245.74,45.0,1,3,2,6.0
1,832,True,True,0,0,0,0,0,1,0,0,True,1.0,1,0,0,0,0,1,0,0,0,-18.07,1,1,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,312.20,3.06,2.41,71.07,27.16,2.266862,2.055465,70,25.596252,8.0,-0.740805,0.671721,0,71.07,27.16,3.06,2.41,0.34,210.70,312.20,53.0,1,1,10,6.0
2,782,False,True,0,0,0,0,0,0,1,0,True,2.0,1,0,0,0,0,1,0,0,0,26.34,0,1,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,221.96,5.77,2.42,48.66,19.11,3.857889,4.290640,70,25.596253,8.0,-0.668612,-0.743612,0,48.66,19.11,5.77,2.42,0.60,140.82,221.96,75.0,1,1,10,7.0
3,732,False,True,0,0,0,1,0,0,0,0,True,2.0,1,0,0,0,0,0,1,0,0,92.47,1,1,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,275.44,4.45,3.20,15.53,25.36,4.429957,0.421875,71,26.852933,8.0,-0.995496,0.094803,0,15.53,25.36,4.45,3.20,0.46,186.22,275.44,108.0,1,2,2,9.0
4,728,True,False,0,0,0,0,0,1,0,0,False,1.0,1,0,0,0,0,1,0,0,0,5.01,0,0,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,157.92,3.90,2.53,29.99,27.12,1.466013,3.613974,71,22.091819,8.0,0.375901,-0.926660,7,29.99,27.12,3.90,2.53,0.44,34.27,157.92,35.0,1,1,10,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23166,183,False,False,0,0,1,0,0,0,0,0,True,1.0,0,0,0,1,0,1,0,0,0,25.43,1,1,0.575847,19.341419,6.736412,4.514974,2.942329,19.341419,8.243270,5.075063,290.23,3.99,3.38,49.57,32.27,3.743865,1.379700,73,27.041779,5.0,-0.938312,0.345790,3,49.57,32.27,3.99,3.38,0.39,287.44,290.23,75.0,4,1,10,8.0
23167,109,True,True,0,0,0,0,0,1,0,0,False,1.0,0,0,0,1,0,1,0,0,0,4.00,0,0,1.151217,25.500196,9.827962,7.474030,2.808932,25.500196,11.779213,7.739310,150.98,4.18,2.95,31.00,30.99,2.027780,3.655203,69,25.861781,5.0,0.485115,-0.874450,0,31.00,30.99,4.18,2.95,0.40,122.97,150.98,35.0,4,1,10,6.0
23168,84,True,True,0,0,0,0,0,1,0,0,False,2.0,0,0,0,1,0,0,0,1,0,4.07,0,0,0.776209,19.454503,8.272970,5.548527,3.595998,19.454503,8.825731,5.317643,41.25,4.67,3.68,39.93,29.20,3.079145,3.511092,69,25.861783,5.0,0.659346,0.751840,0,39.93,29.20,4.67,3.68,0.45,63.11,41.25,44.0,4,3,1,7.0
23169,56,False,True,0,0,0,0,0,1,0,0,False,1.0,0,0,0,1,0,1,0,0,0,3.81,0,0,0.584637,19.113474,8.202171,5.833721,4.090880,19.113474,9.062557,5.568974,41.24,4.23,2.43,81.19,23.75,2.788478,3.180769,71,24.290258,5.0,0.659215,0.751955,0,81.19,23.75,4.23,2.43,0.41,66.34,41.24,85.0,4,1,10,7.0


In [7]:
target

0         8
1         3
2         5
3         2
4         7
         ..
23166     1
23167     4
23168     4
23169     2
23170    11
Name: Yards, Length: 23171, dtype: int64

In [8]:
data.columns

Index(['GameClock_in_seconds', 'in_own_half', 'home_team_has_poss',
       'Formation_ACE', 'Formation_EMPTY', 'Formation_I_FORM',
       'Formation_JUMBO', 'Formation_PISTOL', 'Formation_SHOTGUN',
       'Formation_SINGLEBACK', 'Formation_WILDCAT', 'play_dir_is_left',
       'TimeDelta', 'Quarter_1', 'Quarter_2', 'Quarter_3', 'Quarter_4',
       'Quarter_5', 'Down_1', 'Down_2', 'Down_3', 'Down_4',
       'back_from_scrimmage', 'back_oriented_down_field',
       'back_moving_down_field', 'min_dist', 'max_dist', 'mean_dist',
       'std_dist', 'def_min_dist', 'def_max_dist', 'def_mean_dist',
       'def_std_dist', 'RusherDir', 'RusherS', 'RusherA', 'RusherX', 'RusherY',
       'v_horizontal', 'v_vertical', 'PlayerHeight_dense', 'PlayerAge',
       'WindSpeed_dense', 'Dir_sin', 'Dir_cos', 'diffScoreBeforePlay', 'X',
       'Y', 'S', 'A', 'Dis', 'Orientation', 'Dir', 'YardLine', 'Quarter',
       'Down', 'Distance', 'DefendersInTheBox'],
      dtype='object')

In [9]:
data.isnull().sum()

GameClock_in_seconds        0
in_own_half                 0
home_team_has_poss          0
Formation_ACE               0
Formation_EMPTY             0
Formation_I_FORM            0
Formation_JUMBO             0
Formation_PISTOL            0
Formation_SHOTGUN           0
Formation_SINGLEBACK        0
Formation_WILDCAT           0
play_dir_is_left            0
TimeDelta                   0
Quarter_1                   0
Quarter_2                   0
Quarter_3                   0
Quarter_4                   0
Quarter_5                   0
Down_1                      0
Down_2                      0
Down_3                      0
Down_4                      0
back_from_scrimmage         0
back_oriented_down_field    0
back_moving_down_field      0
min_dist                    0
max_dist                    0
mean_dist                   0
std_dist                    0
def_min_dist                0
def_max_dist                0
def_mean_dist               0
def_std_dist                0
RusherDir 

In [10]:
data_np = data.to_numpy(dtype='float')
target_np = target.to_numpy(dtype='int')
data_np.shape, target_np.shape

((23171, 58), (23171,))

In [0]:
from sklearn.model_selection import train_test_split as ttsplit

X_train, X_test, Y_train, Y_test = ttsplit(data_np, target_np, test_size=0.2)

In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#scaler = MinMaxScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Helpers

In [0]:
def Y_to_199_one_hot(Y):
  y = np.zeros((Y.shape[0], 199))
  for idx, target in enumerate(list(Y)):
      y[idx][target] = 1
  return y

def Y_to_shiftd_199_one_hot(Y):
  y = np.zeros((Y.shape[0], 199))
  for idx, target in enumerate(list(Y)):
      y[idx][target+99] = 1
  return y

In [0]:
from sklearn import metrics
def calc_regression_error(Y_test, Y_test_pred):
  errors = {
      'mean_absolute_error': metrics.mean_absolute_error(Y_test, Y_test_pred),
      'mean_squared_error': metrics.mean_squared_error(Y_test, Y_test_pred),
  }
  return errors

In [0]:
def calc_crps(Y, Y_pred):
  y_true = np.clip(np.cumsum(Y, axis=1), 0, 1)
  y_pred = np.clip(np.cumsum(Y_pred, axis=1), 0, 1)
  val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * Y.shape[0])
  val_s = np.round(val_s, 6)
  return val_s

Neural Net Helpers

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.special import softmax

In [0]:
# Sanity Check
# softmax(np.array([[0.1,0.2,0.5,0.2],[0.3,0.3,0.1,0.1]]), axis=1)

In [0]:
# np.exp(0.3)/(np.exp(0.3) + np.exp(0.3) + np.exp(0.1) + np.exp(0.1))

In [0]:
def eval_crps(net, X, y):
    y_pred = net.predict_proba(X)
    # print("y_pred shape before softmax:", y_pred.shape)
    # print("y_pred before softmax:", y_pred)
    y_pred = softmax(y_pred, axis=1)
    # print("y_pred shape after softmax:", y_pred.shape)
    # print("y_pred after softmax:", y_pred)
    # print("y shape:", y.shape)
    # print("y:", y)
    # print("y_pred sum along axis 1:", np.sum(y_pred, axis=1))
    loss = calc_crps(Y_to_199_one_hot(y), y_pred)
    return loss

In [0]:
def eval_regression(net, X, y):
  y_pred = net.predict_proba(X)
  y_pred = softmax(y_pred, axis=1)
  y_pred = np.sum(np.multiply(np.arange(-99,100), y_pred), axis=1)
  return calc_regression_error(y, y_pred)

Neural Nets

In [0]:
class NFLRegressorNet(nn.Module):
    def __init__(self, Nfeatures):
        super(NFLRegressorNet, self).__init__()
        
        # print("Nfeatures: {}".format(Nfeatures))

        self.net = nn.Sequential(
            nn.Linear(Nfeatures, 1024),
            nn.RReLU(1024),
            nn.Dropout(0.4),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(512),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.RReLU(256),
            nn.Dropout(0.5),
            nn.BatchNorm1d(256),
            nn.Linear(256, 100),
            nn.ReLU(100),
            nn.Dropout(0.4),
            nn.BatchNorm1d(100),
            nn.Linear(100, 1),
        )

       
    def forward(self, x):
      
        batch_size = x.shape[0]
        x = x.float()
        # x = x.view(batch_size, -1)
        out = self.net(x)
        out = out.view(batch_size)
        # print("out:", out)
        # print("out shape:", out.shape)
        return out

In [0]:
class NFLClassifierNet(nn.Module):
    def __init__(self, Nfeatures):
        super(NFLClassifierNet, self).__init__()
        
        # print("Nfeatures: {}".format(Nfeatures))

        self.net = nn.Sequential(
    
            nn.Linear(Nfeatures, 1024),
            nn.ReLU(1024),
            nn.Dropout(0.5),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(512),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(256),
            nn.Dropout(0.5),
            nn.BatchNorm1d(256),
            nn.Linear(256, 199),
            nn.ReLU(199),
            nn.Dropout(0.5),
            nn.BatchNorm1d(199),
            # nn.Softmax(1)
        )

       
    def forward(self, x):
      
        batch_size = x.shape[0]
        x = x.float()
        # x = x.view(batch_size, -1)
        out = self.net(x) # 512
        return out

Skorch Part

In [24]:
!pip install -U skorch

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
from skorch import NeuralNetRegressor, NeuralNetClassifier, NeuralNet
import skorch
from skorch.dataset import CVSplit

Training and Eval

In [0]:
_, Nfeatures = X_train.shape
net = NeuralNet(
        NFLClassifierNet,
        module__Nfeatures=Nfeatures,
        max_epochs=50,
        device='cuda',
        warm_start=True,
        train_split=CVSplit(5, stratified=False),
        lr=0.1,
        optimizer=torch.optim.SGD,
        optimizer__momentum=0.95,
        iterator_train__shuffle=True,
        criterion=torch.nn.CrossEntropyLoss,
        callbacks=[skorch.callbacks.LRScheduler(policy='ReduceLROnPlateau', monitor='valid_loss', mode='min'),
                   skorch.callbacks.EpochScoring(scoring=eval_crps, use_caching=True, name='valid_crps')]
    )

In [27]:
net.fit(X_train.astype(np.float), (Y_train+99).astype(np.long))

Nfeatures: 58
  epoch    train_loss    valid_crps    valid_loss     dur
-------  ------------  ------------  ------------  ------
      1        3.7467        0.0147        3.0018  0.7102
      2        2.9699        0.0141        2.9266  0.6563
      3        2.9311        0.0140        2.9129  0.6454
      4        2.9137        0.0138        2.8821  0.6383
      5        2.8988        0.0138        2.8776  0.6858
      6        2.8914        0.0137        2.8699  0.6434
      7        2.8884        0.0137        2.8674  0.6400
      8        2.8843        0.0137        2.8618  0.6400
      9        2.8805        0.0136        2.8573  0.6336
     10        2.8709        0.0137        2.8576  0.6615
     11        2.8719        0.0137        2.8557  0.6497
     12        2.8672        0.0137        2.8532  0.6526
     13        2.8713        0.0136        2.8501  0.6460
     14        2.8620        0.0135        2.8387  0.6422
     15        2.8607        0.0135        2.8323  0.6655


<class 'skorch.net.NeuralNet'>[initialized](
  module_=NFLClassifierNet(
    (net): Sequential(
      (0): Linear(in_features=58, out_features=1024, bias=True)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.5, inplace=False)
      (3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Linear(in_features=1024, out_features=512, bias=True)
      (5): ReLU(inplace=True)
      (6): Dropout(p=0.5, inplace=False)
      (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Linear(in_features=512, out_features=256, bias=True)
      (9): ReLU(inplace=True)
      (10): Dropout(p=0.5, inplace=False)
      (11): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): Linear(in_features=256, out_features=199, bias=True)
      (13): ReLU(inplace=True)
      (14): Dropout(p=0.5, inplace=False)
      (15): BatchNorm1d(199, eps=1e-05, momentum=0.1, affine=True, track_running_

In [0]:
eval_crps(net, X_test, Y_test+99)

0.013091

In [0]:
eval_crps(net, X_train, Y_train+99)

0.013524

AttributeError: ignored

With KFold Cross Validation

In [0]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, cross_validate, cross_val_predict

In [0]:
_, Nfeatures = X_train.shape
net = NeuralNet(
        NFLClassifierNet,
        module__Nfeatures=Nfeatures,
        max_epochs=1,
        device='cuda',
        warm_start=True,
        train_split=None,
        lr=0.1,
        optimizer=torch.optim.SGD,
        optimizer__momentum=0.95,
        iterator_train__shuffle=True,
        criterion=torch.nn.CrossEntropyLoss,
        callbacks=[skorch.callbacks.LRScheduler(policy='ReduceLROnPlateau', monitor='train_loss', mode='min'),
                  #  skorch.callbacks.EpochScoring(scoring=eval_crps, use_caching=True, name='valid_crps'),
                  ]
    )

In [0]:
kf = KFold(n_splits=5, shuffle=False)
cv_result = cross_validate(net, X_train.astype(np.float), (Y_train+99).astype(np.long), cv=kf, scoring=eval_crps, return_estimator=True)

  epoch    train_loss     dur
-------  ------------  ------
      1        3.7361  0.4188
      2        2.9679  0.3897
      3        2.9270  0.3903
      4        2.9113  0.4121
      5        2.8992  0.4063
      6        2.8904  0.4007
      7        2.8844  0.3842
      8        2.8788  0.3819
      9        2.8748  0.3983
     10        2.8691  0.4136
     11        2.8641  0.3942
     12        2.8618  0.3944
     13        2.8609  0.4191
     14        2.8568  0.4168
     15        2.8518  0.4104
     16        2.8532  0.4004
     17        2.8494  0.4076
     18        2.8445  0.3850
     19        2.8436  0.4067
     20        2.8435  0.4018
     21        2.8463  0.3927
     22        2.8390  0.4007
     23        2.8442  0.3831
     24        2.8335  0.4058
     25        2.8403  0.4354
     26        2.8364  0.3967
     27        2.8358  0.4127
     28        2.8397  0.3973
     29        2.8357  0.4011
     30        2.8328  0.3886
     31        2.8374  0.3964
     32   

In [0]:
cv_result['test_score']

array([0.013883, 0.013575, 0.013413, 0.013414, 0.013196])

In [0]:
net = cv_result['estimator'][np.argmin(cv_result['test_score'])]

In [0]:
eval_crps(net, X_test, Y_test+99)

0.013514

In [0]:
eval_crps(net, X_train, Y_train+99)

0.013243

In [0]:
eval_regression(net, X_test, Y_test)

{'mean_absolute_error': 3.6933778833156405,
 'mean_squared_error': 39.023259815230794}

In [0]:
eval_regression(net, X_train, Y_train)

{'mean_absolute_error': 3.6283886016063582,
 'mean_squared_error': 39.6067135225243}

Regression Net

In [0]:
_, Nfeatures = X_train.shape
net = NeuralNet(
        NFLRegressorNet,
        module__Nfeatures=Nfeatures,
        max_epochs=150,
        device='cuda',
        warm_start=True,
        train_split=CVSplit(5, stratified=False),
        lr=0.1,
        optimizer=torch.optim.SGD,
        optimizer__momentum=0.95,
        iterator_train__shuffle=True,
        criterion=torch.nn.L1Loss,
        callbacks=[skorch.callbacks.LRScheduler(policy='ReduceLROnPlateau', monitor='valid_loss', mode='min'),
                  #  skorch.callbacks.EpochScoring(scoring=eval_crps, use_caching=True, name='valid_crps'),
                  ]
    )

In [0]:
net.fit(X_train.astype(np.float), (Y_train).astype(np.float))

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        3.8399        3.6301  0.4986
      2        3.5340        3.6218  0.5085
      3        3.4858        3.5696  0.4897
      4        3.4645        3.5371  0.5087
      5        3.4538        3.5435  0.4967
      6        3.4506        3.5557  0.4998
      7        3.4408        3.5586  0.4923
      8        3.4329        3.5283  0.5679
      9        3.4159        3.5332  0.4853
     10        3.4445        3.5069  0.4973
     11        3.4146        3.4999  0.5329
     12        3.4080        3.5119  0.5208
     13        3.4095        3.5821  0.5174
     14        3.4051        3.5117  0.5204
     15        3.4131        3.5316  0.4951
     16        3.3842        3.5046  0.5203
     17        3.3949        3.5567  0.4854
     18        3.4066        3.5005  0.5490
     19        3.3700        3.5033  0.5084
     20        3.3587        3.5194  0.5358
     21        3.3616        3.5

<class 'skorch.net.NeuralNet'>[initialized](
  module_=NFLRegressorNet(
    (net): Sequential(
      (0): Linear(in_features=52, out_features=1024, bias=True)
      (1): RReLU(lower=1024, upper=0.3333333333333333)
      (2): Dropout(p=0.4, inplace=False)
      (3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Linear(in_features=1024, out_features=512, bias=True)
      (5): ReLU(inplace=True)
      (6): Dropout(p=0.5, inplace=False)
      (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Linear(in_features=512, out_features=256, bias=True)
      (9): RReLU(lower=256, upper=0.3333333333333333)
      (10): Dropout(p=0.5, inplace=False)
      (11): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): Linear(in_features=256, out_features=100, bias=True)
      (13): ReLU(inplace=True)
      (14): Dropout(p=0.4, inplace=False)
      (15): BatchNorm1d(100, eps=

In [0]:
calc_regression_error(Y_test, net.predict(X_test))

{'mean_absolute_error': 3.4067579181963317,
 'mean_squared_error': 39.01718513361489}